In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import datetime

from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

In [9]:
pages = range(1,101)
# query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시',
#               '갤럭시케이스','안경','모자','하이힐','후드','아이폰케이스','아이폰','자켓','청바지','점퍼','중고차','키보드','노트북',
#               '마우스','목도리','네일','니트','목걸이','나이키','닌텐도','원피스','패딩','바지','향수','냉장고','레고',
#               '반지','셔츠','치마','스피커','정장','선글라스','티셔츠','트레이닝','티비','지갑','세탁기','시계']

query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

# 외제차 + 국산차 : 중고차로 통일
len(query_list)

12

In [10]:
total_df =[]

for query in query_list:
    cat_df = pd.DataFrame()
    
    for page in tqdm_notebook(pages):
        
        pid_list = [] # 아이템 리스트 담기
        bunjang_url = f'https://api.bunjang.co.kr/api/1/find_v2.json?q={query}&order=date&page={page}&stat_device=w&stat_category_required=1&req_ref=search&version=4'
        response = requests.get(bunjang_url.encode('utf-8'))
        
        try:
            item_list = response.json()["list"]
            ids = [item["pid"] for item in item_list]
            pid_list.extend(ids)
        except:
            continue
        
        # 아이템별 데이터 프레임 생성
        df = pd.DataFrame()
        product_id, image_link, title, keyword, cat1, cat2, cat3, view = [],[],[],[],[],[],[],[]
        for pid in pid_list:
            url = f"https://api.bunjang.co.kr/api/1/product/{pid}/detail_info.json?version=4"
            response = requests.get(url)
           
            try : 
                product_id.append(response.json()['item_info']['pid'])
            except : 
                product_id.append(0)
            try :
                image_link.append(response.json()['item_info']['product_image'])
            except : 
                image_link.append(0)
            try : 
                title.append(response.json()['item_info']['name'])
            except : 
                title.append(0)
            try : 
                keyword.append(response.json()['item_info']['keyword'])
            except : 
                keyword.append(0)
            try : 
                cat1.append(list(response.json()['item_info']['category_name'][0].values())[0])
            except : 
                cat1.append(0)
            try : 
                cat2.append(list(response.json()['item_info']['category_name'][1].values())[0])
            except : 
                cat2.append(0)                
            try : 
                cat3.append(list(response.json()['item_info']['category_name'][2].values())[0])
            except : 
                cat3.append(0)
            try : 
                view.append(response.json()['item_info']['num_item_view'])
            except : 
                view.append(0)

        df['product_id'] = product_id
        df['title'] = title
        df['keyword'] = keyword
        df['cat1'] = cat1
        df['cat2'] = cat2
        df['cat3'] = cat3
        df['view'] = view
        df['image'] = image_link
        temp = df
        cat_df = pd.concat([cat_df,temp])

    total_df.append(cat_df)

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_70408/1331840818.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(pages):


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
for i in range(len(total_df)):
    total_df[i].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv',index=False)

In [ ]:
# response.json()['item_info']['pid']
# response.json()['item_info']['product_image']
# response.json()['item_info']['name']
# response.json()['item_info']['num_item_view']
# response.json()['item_info']['keyword']
# list(response.json()['item_info']['category_name'][2].values())[0]